In [ ]:
# 代码1.工业显微相机自动对焦

In [1]:

import cv2
import os
import time
import numpy as np
from typing import Callable, Optional, Tuple
import threading

class CMOSAutoFocusCamera:
    def __init__(
        self,
        camera_id: int = 0,
        save_dir: str = "photos",
        image_format: str = ".jpg",
        window_name: str = "CMOS Camera Control",
        min_pos: int = 0,
        max_pos: int = 15,
        speed: int = 5,
        wait_time: float = 1.5,
        width: int = 1920,
        height: int = 1080,
        fps: int = 30
    ):
        self.camera_id = camera_id
        self.save_dir = save_dir
        self.image_format = image_format
        self.window_name = window_name
        self.min_pos = min_pos
        self.max_pos = max_pos
        self.speed = speed
        self.wait_time = wait_time
        self.width = width
        self.height = height
        self.fps = fps

        self.cap = None
        self.motor = None
        self.is_running = False
        self.preview_thread = None
        self._latest_frame = None

        os.makedirs(self.save_dir, exist_ok=True)

    @staticmethod
    def compute_sharpness(frame) -> float:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
        sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
        return np.sum(sobelx**2 + sobely**2)

    def set_motor(self, motor_controller):
        self.motor = motor_controller
        print("电机已绑定")

    def open_camera(self) -> bool:
        self.cap = cv2.VideoCapture(self.camera_id)
        if not self.cap.isOpened():
            print(f"无法打开相机 {self.camera_id}")
            return False
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, self.width)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, self.height)
        self.cap.set(cv2.CAP_PROP_FPS, self.fps)
        print(f"相机 {self.camera_id} 已打开")
        return True

    def close_camera(self):
        self.is_running = False
        if self.cap:
            self.cap.release()
        cv2.destroyAllWindows()
        print("相机已关闭")

    def _scan_position(self, pos: int) -> Tuple[int, float]:
        if not self.motor:
            raise RuntimeError("电机未绑定！")
        self.motor.absolute_move(pos, self.speed)
        time.sleep(self.wait_time)

        sharpness_sum = 0
        valid = 0
        for _ in range(3):
            ret, frame = self.cap.read()
            if ret:
                sharpness_sum += self.compute_sharpness(frame)
                valid += 1
                self._latest_frame = frame  # 保存最新帧
        avg_sharpness = sharpness_sum / valid if valid > 0 else 0
        print(f"扫描位置 {pos}: 锐度 {avg_sharpness:.2f}")
        return pos, avg_sharpness

    def auto_focus(self):
        """使用贝叶斯优化的智能自动对焦"""
        print("开始贝叶斯优化自动对焦...")
        if not self.cap or not self.cap.isOpened():
            print("错误：相机未打开！")
            return None
        if not self.motor:
            print("错误：电机未绑定！")
            return None

        # 贝叶斯优化参数
        n_initial = 5  # 初始随机采样点数
        n_iterations = 10  # 最大迭代次数
        kappa = 2.0  # UCB的探索参数
        
        # 存储采样历史
        X_sample = []  # 位置
        y_sample = []  # 锐度
        
        # === 第一阶段：初始均匀采样 ===
        print(f"初始采样 {n_initial} 个点...")
        initial_positions = np.linspace(self.min_pos, self.max_pos, n_initial, dtype=int)
        
        for pos in initial_positions:
            current_pos, sharpness = self._scan_position(pos)
            X_sample.append(current_pos)
            y_sample.append(sharpness)
        
        # === 第二阶段：贝叶斯优化迭代 ===
        for iteration in range(n_iterations):
            # 构建高斯过程模型
            X = np.array(X_sample).reshape(-1, 1)
            y = np.array(y_sample)
            
            # 归一化
            y_mean, y_std = y.mean(), y.std()
            if y_std == 0:
                y_std = 1
            y_norm = (y - y_mean) / y_std
            
            # 计算下一个最优采样点（UCB策略）
            next_pos = self._acquisition_ucb(X, y_norm, kappa)
            
            # 确保在范围内
            next_pos = int(np.clip(next_pos, self.min_pos, self.max_pos))
            
            # 如果已经采样过这个点，跳过
            if next_pos in X_sample:
                print(f"迭代 {iteration+1}: 位置 {next_pos} 已采样，跳过")
                continue
            
            # 采样新点
            current_pos, sharpness = self._scan_position(next_pos)
            X_sample.append(current_pos)
            y_sample.append(sharpness)
            
            # 提前停止条件：如果最近3次采样锐度变化很小
            if len(y_sample) >= 3:
                recent_std = np.std(y_sample[-3:])
                if recent_std < 0.01 * np.std(y_sample):
                    print(f"锐度收敛，提前停止")
                    break
        
        # === 找到最佳位置 ===
        best_idx = np.argmax(y_sample)
        best_pos = X_sample[best_idx]
        best_sharp = y_sample[best_idx]
        
        print(f"对焦完成！最佳位置: {best_pos} (锐度: {best_sharp:.2f})")
        print(f"总共采样 {len(X_sample)} 个点")
        
        # === 移动到最佳位置并保存照片 ===
        self.motor.absolute_move(best_pos, self.speed)
        time.sleep(self.wait_time + 1)
        
        ret, frame = self.cap.read()
        if ret:
            path = os.path.join(self.save_dir, f"best_{time.strftime('%Y%m%d_%H%M%S')}.jpg")
            cv2.imwrite(path, frame)
            print(f"最佳照片已保存: {path}")
            self._latest_frame = frame
            return best_pos, path
        else:
            print("警告：无法捕获最终图像")
            return best_pos, None

    def _acquisition_ucb(self, X_sample, y_sample, kappa):
        """
        Upper Confidence Bound (UCB) 采集函数
        返回下一个最优采样位置
        """
        # 创建候选位置网格（整数位置）
        X_test = np.arange(self.min_pos, self.max_pos + 1).reshape(-1, 1)
        
        # 计算每个候选点的均值和标准差
        mu = np.zeros(len(X_test))
        sigma = np.zeros(len(X_test))
        
        for i, x in enumerate(X_test):
            # 使用高斯过程预测
            mu[i], sigma[i] = self._gp_predict(x, X_sample, y_sample)
        
        # UCB = 均值 + kappa * 标准差
        ucb = mu + kappa * sigma
        
        # 返回UCB最大的位置
        best_idx = np.argmax(ucb)
        return X_test[best_idx][0]

    def _gp_predict(self, x_new, X_sample, y_sample, length_scale=None):
        """
        简化的高斯过程预测
        返回 (均值, 标准差)
        """
        if length_scale is None:
            # 自适应长度尺度（约为搜索范围的1/4）
            length_scale = (self.max_pos - self.min_pos) / 4
        
        X_sample = X_sample.reshape(-1, 1)
        y_sample = np.array(y_sample).flatten()  # 确保是1D数组
        x_new = np.atleast_2d(x_new)
        
        # RBF核函数
        def rbf_kernel(x1, x2, l=length_scale):
            dist = np.sum((x1 - x2) ** 2, axis=1)
            return np.exp(-dist / (2 * l ** 2))
        
        # 计算核矩阵
        K = np.zeros((len(X_sample), len(X_sample)))
        for i in range(len(X_sample)):
            for j in range(len(X_sample)):
                K[i, j] = rbf_kernel(X_sample[i:i+1], X_sample[j:j+1])
        
        # 添加噪声项防止奇异
        K += 1e-6 * np.eye(len(X_sample))
        
        # 计算 k* (新点与已知点的协方差)
        k_star = np.array([rbf_kernel(x_new, X_sample[i:i+1]) for i in range(len(X_sample))]).flatten()
        
        # 预测均值
        try:
            K_inv = np.linalg.inv(K)
            mu = k_star @ K_inv @ y_sample  # 现在维度匹配了
        except np.linalg.LinAlgError:
            mu = np.mean(y_sample)
        
        # 预测标准差
        k_star_star = rbf_kernel(x_new, x_new)[0]
        try:
            sigma = np.sqrt(max(0, k_star_star - k_star @ K_inv @ k_star))
        except:
            sigma = 1.0
        
        return float(mu), float(sigma)

    def capture_photo(self, prefix: str = "photo"):
        ret, frame = self.cap.read()
        if not ret:
            print("拍照失败")
            return None
        path = os.path.join(self.save_dir, f"{prefix}_{time.strftime('%Y%m%d_%H%M%S')}.jpg")
        cv2.imwrite(path, frame)
        print(f"照片已保存: {path}")
        return path

    # ==================== 核心升级：灵活运行模式 ====================
    def run(
        self,
        *,
        show_preview: bool = True,
        on_key_s: Callable = None,
        on_key_a: Callable = None,
        block: bool = True
    ):
        """
        灵活运行相机

        参数：
            show_preview: 是否显示预览窗口
            on_key_s: 按 s 键时执行的函数（默认拍照）
            on_key_a: 按 a 键时执行的函数（默认自动对焦）
            block: 是否阻塞主线程（False 时后台运行）
        """
        if not self.open_camera():
            return

        # 默认行为
        if on_key_s is None:
            on_key_s = self.capture_photo
        if on_key_a is None:
            on_key_a = self.auto_focus

        self.is_running = True

        def _preview_loop():
            print("按 s=执行自定义动作, a=自动对焦, q=退出")
            while self.is_running:
                ret, frame = self.cap.read()
                if not ret:
                    break

                if show_preview:
                    cv2.imshow(self.window_name, frame)

                key = cv2.waitKey(1) & 0xFF
                if key == ord('s'):
                    on_key_s()
                elif key == ord('a'):
                    on_key_a()
                elif key == ord('q'):
                    break

            self.close_camera()

        if block:
            _preview_loop()
        else:
            self.preview_thread = threading.Thread(target=_preview_loop, daemon=True)
            self.preview_thread.start()
            print("相机已在后台运行（无阻塞）")

    # ==================== 便捷快捷方式 ====================
    def run_interactive(self):
        """传统交互模式：显示窗口 + s拍照 + a对焦"""
        self.run(show_preview=True)

    def run_headless(self):
        """无窗口模式：仅后台运行，可手动调用拍照/对焦"""
        self.run(show_preview=False, block=False)
        print("无界面模式启动成功！可调用 cam.capture_photo() 或 cam.auto_focus()")

    def stop(self, wait=True):
        self.is_running = False
        if self.preview_thread and wait:
            self.preview_thread.join(timeout=2.0)
        self.close_camera()
        print("相机已安全关闭")

    def __enter__(self):
        self.run_headless()
        time.sleep(1)
        return self

    def __exit__(self, *args):
        self.stop()

# ==================== 测试代码 ====================
if __name__ == "__main__":
    print("CMOSAutoFocusCamera 已加载！")

CMOSAutoFocusCamera 已加载！


In [ ]:
# 代码2.傅里叶变换高频信息提取超参数优化

In [ ]:
import cv2
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
import re

def get_image_energy_spectrum(image_path, target_size=(1024, 1024)):
    """计算图像能量谱，增加错误打印"""
    try:
        # 使用 np.fromfile 配合 cv2.imdecode 完美兼容中文路径
        img_array = np.fromfile(image_path, dtype=np.uint8)
        img = cv2.imdecode(img_array, cv2.IMREAD_GRAYSCALE)
        
        if img is None:
            print(f"  [警告] 无法解码图片: {image_path}")
            return None
            
        img = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)
        # 执行FFT并中心化
        f_shift = np.fft.fftshift(np.fft.fft2(img))
        return np.abs(f_shift)**2
    except Exception as e:
        print(f"  [错误] 处理图片 {os.path.basename(image_path)} 时出错: {e}")
        return None

def compute_metrics(spec, mask):
    """计算单张图的三个指标"""
    total_energy = np.sum(spec)
    high_freq_part = spec[~mask]
    high_energy = np.sum(high_freq_part)
    
    # 1. 占比
    ratio = high_energy / total_energy if total_energy > 0 else 0
    # 2. 绝对能量 (log)
    abs_energy = np.log1p(high_energy)
    # 3. 谱熵
    if high_energy > 0:
        p = high_freq_part / high_energy
        p = p[p > 0]
        entropy = -np.sum(p * np.log2(p))
    else:
        entropy = 0
    return ratio, abs_energy, entropy

def analyze_all_images(input_folder):
    # 1. 载入数据
    exts = ["*.jpg", "*.png", "*.jpeg", "*.JPG", "*.PNG"]
    files = []
    for e in exts:
        files.extend(glob.glob(os.path.join(input_folder, e)))
    
    all_data = []
    standard_size = (1024, 1024)
    h, w = standard_size
    y, x = np.ogrid[-h//2:h//2, -w//2:w//2]
    dist_matrix = np.sqrt(x**2 + y**2)

    print(f"开始载入图片，共发现 {len(files)} 个文件...")
    for f in files:
        # 提取文件名中的目数
        nums = re.findall(r'\d+', os.path.basename(f))
        if not nums:
            print(f"  [跳过] 文件名中未发现数字: {os.path.basename(f)}")
            continue
            
        grit = int(nums[0])
        spec = get_image_energy_spectrum(f, standard_size)
        if spec is not None:
            all_data.append({'grit': grit, 'spec': spec, 'name': os.path.basename(f)})
            print(f"  [成功] 已加载: {os.path.basename(f)} (目数: {grit})")

    if len(all_data) < 3:
        print(f"\n[错误] 有效图片张数仅为 {len(all_data)}，无法进行相关性分析。")
        print("请检查图片是否损坏或文件名是否包含数字目数。")
        return

    # 按目数升序排序
    all_data.sort(key=lambda x: x['grit'])
    grits = np.array([d['grit'] for d in all_data])
    print(f"\n数据载入完成，共 {len(all_data)} 张有效图片，即将开始分析全序列趋势...")

    # 2. 扫描与分析
    metrics_labels = ["Ratio (占比)", "AbsEnergy (绝对能量)", "Entropy (谱熵)"]
    radii = np.linspace(1, 150, 75)
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    best_params = []

    for m_idx, m_name in enumerate(metrics_labels):
        corrs = []
        for r in radii:
            mask = dist_matrix <= r
            vals = [compute_metrics(d['spec'], mask)[m_idx] for d in all_data]
            c, _ = spearmanr(grits, vals)
            corrs.append(c)
        
        corrs = np.array(corrs)
        best_i = np.nanargmin(corrs)
        best_r = radii[best_i]
        best_c = corrs[best_i]
        best_params.append((best_r, best_c))

        # 绘图
        axes[m_idx].plot(radii, corrs, color='blue', lw=1.5)
        axes[m_idx].axvline(best_r, color='red', linestyle='--', label=f'Best R={best_r:.1f}')
        axes[m_idx].set_title(f"{m_name}\nBest Corr: {best_c:.4f}", fontsize=12)
        axes[m_idx].set_xlabel("Mask Radius")
        axes[m_idx].set_ylabel("Spearman Correlation")
        axes[m_idx].grid(True, alpha=0.3)
        axes[m_idx].legend()

    plt.tight_layout()
    plt.show()

    # 3. 输出全样本排名
    print("\n" + "="*85)
    print(f"{'指标类型':<15} | {'最佳半径':<10} | {'全序列相关度':<12}")
    print("-" * 85)
    for i, m_name in enumerate(metrics_labels):
        print(f"{m_name:<15} | {best_params[i][0]:>8.2f} | {best_params[i][1]:>12.4f}")
    
    # 详细排名（以Ratio为例）
    print("\n" + "█" * 25 + " [基于 Ratio 指标的全样本粗糙度排名] " + "█" * 25)
    best_r_ratio = best_params[0][0]
    final_mask = dist_matrix <= best_r_ratio
    
    ranking = []
    for d in all_data:
        val = compute_metrics(d['spec'], final_mask)[0]
        ranking.append((d['name'], d['grit'], val))
    
    # 指标越大越粗糙
    ranking.sort(key=lambda x: x[2], reverse=True)
    
    print(f"{'排名':<5} | {'文件名':<20} | {'物理目数':<10} | {'计算数值 (指标越大越粗糙)'}")
    print("-" * 80)
    for i, item in enumerate(ranking, 1):
        print(f"{i:<5} | {item[0]:<20} | {item[1]:<10} | {item[2]:.6e}")
    print("=" * 85)

if __name__ == "__main__":
    # 请填入您的文件夹路径
    path = r"/home/zsy/photoprocess/photo" 
    if os.path.exists(path):
        analyze_all_images(path)
    else:
        print(f"错误：找不到路径 {path}")

In [ ]:
# 代码3.除1交叉验证

In [ ]:
import cv2
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
import re

# ================= 1. 核心处理函数 =================

def get_spectra(image_path, target_size=(1024, 1024)):
    """获取原始和预处理后的频谱"""
    try:
        # 兼容中文路径
        img_array = np.fromfile(image_path, dtype=np.uint8)
        img = cv2.imdecode(img_array, cv2.IMREAD_GRAYSCALE)
        if img is None: return None, None
        img = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)

        # 1. Raw Spectrum (原始)
        f_raw = np.fft.fftshift(np.fft.fft2(img))
        spec_raw = np.abs(f_raw)**2

        # 2. Normalized Spectrum (CLAHE + Z-Score)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        img_pre = clahe.apply(img).astype(np.float32)
        img_norm = (img_pre - np.mean(img_pre)) / (np.std(img_pre) + 1e-5)
        f_norm = np.fft.fftshift(np.fft.fft2(img_norm))
        spec_norm = np.abs(f_norm)**2

        return spec_raw, spec_norm
    except: return None, None

def compute_metrics(spec, mask):
    """计算特征指标"""
    total = np.sum(spec)
    high_freq_part = spec[~mask]
    high_energy = np.sum(high_freq_part)
    ratio = high_energy / total if total > 0 else 0
    abs_e = np.log1p(high_energy)
    entropy = 0
    if high_energy > 0:
        p = high_freq_part / high_energy
        p = p[p > 0]
        entropy = -np.sum(p * np.log2(p))
    return ratio, abs_e, entropy

# ================= 2. 全集合分析引擎 =================

def analyze_all_images_together(input_folder):
    # 加载所有图片
    exts = ["*.jpg", "*.png", "*.jpeg", "*.JPG", "*.PNG"]
    files = []
    for e in exts: files.extend(glob.glob(os.path.join(input_folder, e)))
    
    data = []
    standard_size = (1024, 1024)
    y, x = np.ogrid[-512:512, -512:512]
    dist_matrix = np.sqrt(x**2 + y**2)

    print(f"正在载入并预处理全集合图像，共 {len(files)} 张...")
    for f in files:
        nums = re.findall(r'\d+', os.path.basename(f))
        if nums:
            grit = int(nums[0])
            s_raw, s_norm = get_spectra(f, standard_size)
            if s_raw is not None:
                data.append({'grit': grit, 'spec_raw': s_raw, 'spec_norm': s_norm, 'name': os.path.basename(f)})

    # 按目数升序排序
    data.sort(key=lambda x: x['grit'])
    grits = [d['grit'] for d in data]
    n_samples = len(data)
    
    metrics_names = ["Ratio", "AbsEnergy", "Entropy"]
    modes = ["Raw (原始)", "Normalized (预处理)"]
    radii_space = np.linspace(2, 150, 75)
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    plt.subplots_adjust(hspace=0.4, wspace=0.3)

    print(f"\n" + "█"*110)
    print(f"  全集合综合分析报告 (样本数: {n_samples})")
    print("█"*110)

    for mode_idx, mode_label in enumerate(modes):
        spec_key = 'spec_raw' if mode_idx == 0 else 'spec_norm'
        print(f"\n>>> 模式: {mode_label}")

        for m_idx, m_name in enumerate(metrics_names):
            # --- 1. 全集合扫描找最佳半径 (Max |r|) ---
            all_corrs_orig = []
            for r in radii_space:
                mask = dist_matrix <= r
                vals = [compute_metrics(d[spec_key], mask)[m_idx] for d in data]
                c, _ = spearmanr(grits, vals)
                all_corrs_orig.append(c)
            
            all_corrs_abs = np.abs(all_corrs_orig)
            best_idx = np.nanargmax(all_corrs_abs)
            best_r = radii_space[best_idx]
            best_c_orig = all_corrs_orig[best_idx]
            
            # --- 2. 留一法交叉验证 (LOOCV) ---
            loocv_radii = []
            for i in range(n_samples):
                train_data = data[:i] + data[i+1:]
                train_grits = [d['grit'] for d in train_data]
                
                fold_best_r, fold_max_abs_c = -1, 0
                for r in radii_space:
                    mask = dist_matrix <= r
                    vals = [compute_metrics(d[spec_key], mask)[m_idx] for d in train_data]
                    c_abs = abs(spearmanr(train_grits, vals)[0])
                    if c_abs > fold_max_abs_c:
                        fold_max_abs_c, fold_best_r = c_abs, r
                loocv_radii.append(fold_best_r)
            
            std_r = np.std(loocv_radii)

            # --- 3. 打印详细排名 (基于最佳半径) ---
            print(f"\n  【{m_name}】 最佳半径: {best_r:.1f} | |r|: {abs(best_c_orig):.4f} | LOOCV Std: {std_r:.2f}")
            
            best_mask = dist_matrix <= best_r
            rank_list = []
            for d in data:
                val = compute_metrics(d[spec_key], best_mask)[m_idx]
                rank_list.append((d['name'], d['grit'], val))
            
            # 排序逻辑：如果相关性为负，数值降序；如果为正，数值升序。输出始终是从粗糙到光滑。
            rank_list.sort(key=lambda x: x[2], reverse=(best_c_orig < 0))
            
            print(f"    {'排名':<5} | {'文件名':<20} | {'目数':<8} | {'指标数值'}")
            print(f"    {'-' * 55}")
            for i, res in enumerate(rank_list, 1):
                # 标记乱序：如果当前目数比前一个目数小，说明是异常点
                print(f"    {i:<5} | {res[0]:<20} | {res[1]:<8} | {res[2]:.4e}")

            # --- 4. 可视化绘图 ---
            ax = axes[mode_idx, m_idx]
            ax.plot(radii_space, all_corrs_abs, color='blue', alpha=0.7, label='Abs Correlation')
            ax.axvline(best_r, color='red', linestyle='--', label=f'Best R={best_r:.1f}')
            # 底部画LOOCV稳定性点
            ax.scatter(loocv_radii, [0.05]*len(loocv_radii), color='red', marker='x', s=20, alpha=0.6, label='LOOCV Points')
            
            ax.set_title(f"{mode_label}-{m_name}\nStd: {std_r:.1f} | |r|: {abs(best_c_orig):.4f}", fontsize=10)
            ax.set_ylim(0, 1.1)
            ax.grid(True, alpha=0.2)
            if mode_idx == 1: ax.set_xlabel("Mask Radius")
            if m_idx == 0: ax.set_ylabel("|Spearman|")
            ax.legend(fontsize=7, loc='lower right')

    plt.suptitle("Global Analysis of All 17 Samples (60-10000 Grit)", fontsize=16, y=0.98)
    plt.show()

if __name__ == "__main__":
    path = r"/home/zsy/photoprocess/photo" # 替换为你的路径
    if os.path.exists(path):
        analyze_all_images_together(path)
    else:
        print("找不到路径！")

In [ ]:
# 代码4.使用SAM液滴接触角的自动计算

In [ ]:
import cv2
import numpy as np
from ultralytics import SAM
import matplotlib.pyplot as plt
import os
import pandas as pd  # 新增：用于数据统计

def analyze_single_image(image_path, output_path, model):
    """处理单张图片并返回接触角，同时保存结果图"""
    try:
        img = cv2.imread(image_path)
        if img is None: return None
        
        # 预处理与增强
        lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        l, a, b = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
        l = clahe.apply(l)
        img_enhanced = cv2.cvtColor(cv2.merge((l,a,b)), cv2.COLOR_LAB2BGR)
        h, w, _ = img.shape

        # 1. 自动定位提示点 (寻找反光点)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(gray, 210, 255, cv2.THRESH_BINARY)
        M = cv2.moments(thresh)
        if M["m00"] > 50:
            cx, cy = int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"])
        else:
            cx, cy = w // 2, h // 2 - 20

        # 2. SAM 推理
        results = model.predict(img_enhanced, points=[[cx, cy]], labels=[1], verbose=False)
        if not results[0].masks: return None

        mask = results[0].masks.data[0].cpu().numpy()
        mask = cv2.resize((mask > 0).astype(np.uint8), (w, h))

        # 3. 提取轮廓与拟合
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        cnt = max(contours, key=cv2.contourArea)
        baseline_y = np.max(cnt[:, 0, 1]) - 2
        y_min = np.min(cnt[:, 0, 1])
        total_h = baseline_y - y_min
        
        # 过滤拟合点 (取中间 70% 区域)
        fit_points = cnt[(cnt[:, 0, 1] < baseline_y - 0.15 * total_h) & 
                         (cnt[:, 0, 1] > y_min + 0.15 * total_h)][:, 0, :]

        # 最小二乘圆拟合
        x_f, y_f = fit_points[:, 0], fit_points[:, 1]
        A = np.column_stack([x_f, y_f, np.ones(len(x_f))])
        b_val = x_f**2 + y_f**2
        c, _, _, _ = np.linalg.lstsq(A, b_val, rcond=None)
        xc, yc = c[0]/2, c[1]/2
        r = np.sqrt(c[2] + xc**2 + yc**2)

        # 4. 计算角度
        dy = abs(baseline_y - yc)
        dx = np.sqrt(max(0, r**2 - dy**2))
        theta_rad = np.arctan2(dx, dy)
        theta_deg = np.degrees(theta_rad) if yc < baseline_y else 180 - np.degrees(theta_rad)

        # 5. 绘图并保存
        plt.figure(figsize=(10, 6))
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.plot(cnt[:,0,0], cnt[:,0,1], 'lime', linewidth=1, label='SAM Contour')
        t = np.linspace(0, 2*np.pi, 200)
        plt.plot(xc + r*np.cos(t), yc + r*np.sin(t), 'r--', label='Circle Fit')
        plt.axhline(y=baseline_y, color='yellow', linewidth=1)
        plt.title(f"Angle: {theta_deg:.2f}°")
        plt.axis('off')
        plt.savefig(output_path, bbox_inches='tight', dpi=100)
        plt.close()
        
        return theta_deg
    except:
        return None

def batch_process(input_dir, output_dir):
    if not os.path.exists(output_dir): os.makedirs(output_dir)
    model = SAM("sam_b.pt")
    
    results_list = []
    files = [f for f in os.listdir(input_dir) if f.endswith(('.jpg', '.png'))]
    
    print(f"开始批量处理 {len(files)} 张图片...")

    for filename in files:
        try:
            # 解析文件名: x-y.jpg -> x 是目数
            mesh_str = filename.split('-')[0]
            mesh_num = int(mesh_str)
            
            img_path = os.path.join(input_dir, filename)
            res_path = os.path.join(output_dir, f"res_{filename}")
            
            angle = analyze_single_image(img_path, res_path, model)
            
            if angle is not None:
                results_list.append({'Mesh': mesh_num, 'Angle': angle})
                print(f"成功处理: {filename} -> {angle:.2f}°")
            else:
                print(f"失败跳过: {filename}")
        except Exception as e:
            print(f"解析文件名出错 {filename}: {e}")

    # --- 统计与绘图 ---
    if not results_list:
        print("没有成功获取到任何数据。")
        return

    df = pd.DataFrame(results_list)
    
    # 按目数分组计算平均值和标准差
    stats = df.groupby('Mesh')['Angle'].agg(['mean', 'std', 'count']).reset_index()
    stats = stats.sort_values('Mesh') # 确保X轴按目数从小到大排序
    
    print("\n统计结果：")
    print(stats)

    # 绘制关系图
    plt.figure(figsize=(8, 6))
    plt.errorbar(stats['Mesh'], stats['mean'], yerr=stats['std'], 
                 fmt='o-', color='blue', ecolor='red', capsize=5, 
                 linewidth=2, markersize=8, label='Mean ± SD')
    
    plt.xlabel('Substrate Mesh Number (目数)', fontsize=12)
    plt.ylabel('Average Contact Angle (°)', fontsize=12)
    plt.title('Relationship between Substrate Mesh and Contact Angle', fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend()
    
    # 保存统计图
    plt.savefig(os.path.join(output_dir, "summary_plot.png"), dpi=300)
    plt.show()

# 设置路径并运行
input_folder = r"/home/zsy/CA/CA_photo"
output_folder = r"/home/zsy/CA/CA_results"
batch_process(input_folder, output_folder)

In [ ]:
# 代码5.手动标记数据

In [ ]:
import cv2
import numpy as np
import os

# 全局变量
points = []
current_img = None
img_display = None

def click_event(event, x, y, flags, param):
    global points, img_display
    if event == cv2.EVENT_LBUTTONDOWN:
        # 记录坐标
        points.append((x, y))
        
        # 实时显示：前3个红点，后2个黄点
        color = (0, 0, 255) if len(points) <= 3 else (0, 255, 255)
        cv2.circle(img_display, (x, y), 5, color, -1)
        
        # 实时标注编号
        cv2.putText(img_display, str(len(points)), (x + 10, y - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        cv2.imshow("Manual Mark", img_display)

def solve_circle_3pts(p1, p2, p3):
    """根据三个点坐标通过几何法解圆心和半径"""
    x1, y1 = p1; x2, y2 = p2; x3, y3 = p3
    D = 2 * (x1 * (y2 - y3) + x2 * (y3 - y1) + x3 * (y1 - y2))
    if D == 0: return None
    xc = ((x1**2 + y1**2) * (y2 - y3) + (x2**2 + y2**2) * (y3 - y1) + (x3**2 + y3**2) * (y1 - y2)) / D
    yc = ((x1**2 + y1**2) * (x3 - x2) + (x2**2 + y2**2) * (x1 - x3) + (x3**2 + y3**2) * (x2 - x1)) / D
    r = np.sqrt((x1 - xc)**2 + (y1 - yc)**2)
    return xc, yc, r

def run_manual_marking(input_dir, output_dir):
    global points, current_img, img_display
    
    if not os.path.exists(output_dir): os.makedirs(output_dir)
    files = [f for f in os.listdir(input_dir) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
    
    print("--- 人工标记说明 ---")
    print("1. 依次点击液滴圆弧上的 3 个点 (红色)")
    print("2. 依次点击基准线(表面)上的 2 个点 (黄色)")
    print("3. 按 'c' 键计算并保存，按 'r' 重置，按 'q' 退出")

    for filename in files:
        points = []
        img_path = os.path.join(input_dir, filename)
        current_img = cv2.imread(img_path)
        img_display = current_img.copy()
        
        cv2.imshow("Manual Mark", img_display)
        cv2.setMouseCallback("Manual Mark", click_event)
        
        while True:
            key = cv2.waitKey(1) & 0xFF
            if key == ord('r'): # 重置当前图片
                points = []
                img_display = current_img.copy()
                cv2.imshow("Manual Mark", img_display)
            elif key == ord('q'): # 退出程序
                cv2.destroyAllWindows()
                return
            elif key == ord('c') and len(points) == 5: # 计算并保存
                # --- 1. 计算拟合数据 ---
                xc, yc, r = solve_circle_3pts(points[0], points[1], points[2])
                p4, p5 = points[3], points[4]
                
                # 基准线平均高度
                baseline_y_avg = (p4[1] + p5[1]) / 2
                dy = abs(baseline_y_avg - yc)
                dx = np.sqrt(max(0, r**2 - dy**2))
                
                # 计算接触角
                theta_rad = np.arctan2(dx, dy)
                theta_deg = np.degrees(theta_rad) if yc < baseline_y_avg else 180 - np.degrees(theta_rad)
                
                # --- 2. 绘制保存用的图片 (包含所有标记点) ---
                res_plot = current_img.copy()
                
                # A. 绘制 5 个原始标记点和编号
                for i, pt in enumerate(points):
                    dot_color = (0, 0, 255) if i < 3 else (0, 255, 255) # 前3红后2黄
                    cv2.circle(res_plot, pt, 6, dot_color, -1) # 画实心点
                    cv2.putText(res_plot, str(i+1), (pt[0]+10, pt[1]-10), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, dot_color, 2)
                
                # B. 绘制拟合圆 (红色虚线感)
                cv2.circle(res_plot, (int(xc), int(yc)), int(r), (0, 0, 255), 2)
                
                # C. 绘制拟合基线 (延长线，黄色)
                # 为了美观，画一条贯穿左右标记点的长线
                ext_len = 100
                cv2.line(res_plot, (p4[0]-ext_len, p4[1]), (p5[0]+ext_len, p5[1]), (0, 255, 255), 2)
                
                # D. 标注接触角数值
                text_bg = (int(points[1][0]), int(points[1][1] - 50)) # 在液滴上方显示
                cv2.putText(res_plot, f"CA: {theta_deg:.2f} deg", (50, 60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)
                
                # --- 3. 保存 ---
                save_path = os.path.join(output_dir, f"manual_marked_{filename}")
                cv2.imwrite(save_path, res_plot)
                print(f"已保存包含标记点的图片: {filename} | 角度: {theta_deg:.2f}°")
                break
                
        if key == ord('q'): break

    cv2.destroyAllWindows()
    print("所有图片处理完成！")

# --- 设置路径运行 ---
# 请确保文件夹路径存在
input_folder = r"C:\Users\Admin\Desktop\CA\CA_photo"
output_folder = r"C:\Users\Admin\Desktop\CA\CA_manual_results"

run_manual_marking(input_folder, output_folder)